### Load packages

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
from sklearn import cross_validation as cv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model

### Load csv

In [48]:
motionloc = 'C:/Users/Valued Customer/Desktop/motion_feature.csv'
nomotionloc = 'C:/Users/Valued Customer/Desktop/nomotion_feature.csv'
motion = pd.read_csv(motionloc, header=None,skip_blank_lines=True) # load whole table, row 22 is NA
nomotion = pd.read_csv(nomotionloc, header=None,skip_blank_lines=True)
motion_data = motion.ix[1:,2:]  # column 2-end are features, row 0 is column name
nomotion_data = nomotion.ix[1:,2:]
motion_class = np.empty(len(motion)-1) 
motion_class [:] = 1 #
#motion.ix[1:,7] # column 7 is class
nomotion_class = np.empty(len(nomotion)-1) 
nomotion_class [:] = 0 
#nomotion.ix[1:,7]
x=motion_data.append(nomotion_data,ignore_index = True)
x=x.astype('float64') # convert string to float
y=pd.concat([pd.DataFrame(motion_class),pd.DataFrame(nomotion_class)],ignore_index = True)
for i in range(len(y)):
    y.ix[i]=y.ix[i].astype('category') # convert string to categorical

### Exploratory analysis

In [3]:
# data_all=x
# data_all['class']=y

In [4]:
# _=data_all.boxplot(by='class')

In [5]:
# _=data_all.groupby('class').boxplot() 

In [6]:
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')

In [7]:
# _=data_all.hist()

### Split training and testing

In [49]:
x_train, x_test, y_train, y_test=cv.train_test_split(x,y,test_size=0.1)

### Data preprocessing

In [50]:
x_train_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_test_scaled= scaler.transform(x_test)

In [51]:
data_scaled=pd.DataFrame(x_train_scaled)
data_scaled['class']=y_train

In [16]:
# data_all=x
# data_all['class']=y
# _=data_all.boxplot(by='class')
# _=data_all.groupby('class').boxplot() 
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')
# _=data_all.hist()

### Build classifier

In [52]:
names=['LDA','QDA','logistic regression','random forest','SVC','KNN']
classifiers=[LinearDiscriminantAnalysis(),
             QuadraticDiscriminantAnalysis(),
             LogisticRegression(),
             RandomForestClassifier(max_depth=5, n_estimators=3, max_features=1),
             SVC(gamma=2, C=1),
             KNeighborsClassifier(3)]
for name, clf in zip(names,classifiers):
    #clf.fit(x_train_scaled,y_train)
    #print(clf.score(x_test_scaled,y_test))
    score = cross_validation.cross_val_score(clf, x_train_scaled, y_train[0], cv=3)#,scoring=scoring)
    print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

LDA: accuracy 0.718670076726+/-0.032552230336
QDA: accuracy 0.670147769253+/-0.0463388237748
logistic regression: accuracy 0.772023302074+/-0.0440155405685
random forest: accuracy 0.776641091219+/-0.00843946712414
SVC: accuracy 0.81059960216+/-0.0317961271787
KNN: accuracy 0.786445012788+/-0.0359120328556


### Build Classifier with first 5 of the features

In [62]:
for name, clf in zip(names,classifiers):
    #clf.fit(x_train_scaled,y_train)
    #print(clf.score(x_test_scaled,y_test))
    score = cross_validation.cross_val_score(clf, x_train_scaled[:,0:5], y_train[0], cv=3)#,scoring=scoring)
    print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

LDA: accuracy 0.742824666098+/-0.0333850279914
QDA: accuracy 0.631500426257+/-0.0654475264685
logistic regression: accuracy 0.757388462631+/-0.0442474725725
random forest: accuracy 0.752486501847+/-0.0104720237412
SVC: accuracy 0.815501562944+/-0.0183673059348
KNN: accuracy 0.786231884058+/-0.0311678451704


### Lasso

In [54]:
lasso = linear_model.Lasso()
alphas = np.logspace(-4, -.5, 10)
scores=[]
scores_std=[]
for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_validation.cross_val_score(lasso, x_train_scaled, y_train[0], n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

In [55]:
scores

[0.12247101587134453,
 0.12499952239998464,
 0.13056723516160462,
 0.14303161534047781,
 0.15786759912533257,
 0.16370656588248067,
 0.1693190333713944,
 0.15360408027915304,
 0.096007492340691877,
 -0.014695127389413587]

### Ridge

In [58]:
n_alphas = 10
alphas = np.logspace(-10, -1, n_alphas)
clf = linear_model.Ridge(fit_intercept=False)
scores =[]
for a in alphas:
    clf.set_params(alpha=a)
    this_scores = cross_validation.cross_val_score(clf, x_train_scaled, y_train[0], cv=3)#,scoring=scoring)
    scores.append(np.mean(this_scores))

In [59]:
scores

[-1.7115082900394178,
 -1.7115082538080715,
 -1.7115078915033226,
 -1.7115042686076578,
 -1.7114680553568837,
 -1.711107486765453,
 -1.7076518882110043,
 -1.6836337400460362,
 -1.6389254246671783,
 -1.6149058785517372]